In [47]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tabulate import tabulate
from tqdm import tqdm_notebook

In [48]:
from tensorflow.keras.datasets import mnist

In [49]:
# Определение функций активации
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

In [59]:
# Загрузка данных MNIST 70_000 x 784
(X_train, y_train), (X_test, y_test) = mnist.load_data()
y_train = y_train[:6_000]
y_test = y_test[:2_000]
# Преобразование данных в нужный формат
X_train = X_train.reshape(60_000, 784)[:6_000]
X_test = X_test.reshape(10_000, 784)[:2_000]
X_train = X_train / 255
X_test = X_test / 255
y_train_onehot = np.zeros((y_train.size, y_train.max() + 1))
y_train_onehot[np.arange(y_train.size), y_train] = 1

In [61]:
print(X_train.shape, X_test.shape)

(6000, 784) (2000, 784)


In [62]:
print(y_train.shape, y_test.shape)

(6000,) (2000,)


In [60]:
# Best n_hidden: 522
# Best learning_rate: 0.1
# Best n_epochs: 1500
# Best batch_size: 64
# Best accuracy: 0.9550

# Определение параметров модели
n_inputs = X_train.shape[1]   # Количество входных нейронов
n_outputs = len(np.unique(y_train))  # Количество выходных нейронов
n_hidden = 522 # np.ceil((n_inputs + n_outputs) / 2).astype(int)  # Количество нейронов в скрытом слое
learning_rate = 0.1 # Скорость обучения
n_epochs = 501 # количество эпох
batch_size = 32 # количество образцов данных


Переменная batch_size определяет количество образцов данных, которые будут использоваться за один раз для обновления весов модели в процессе обучения.
Вместо того, что бы использовать все обучающие данные одновременно для каждого шага обновления,
мы используем мини-батчи (батчи) данных для более эффективного вычисления градиента и обновления весов.

Слишком маленькое значение может привести к тому, что обновление весов будет недостаточно эффективным,
из-за того, что маленькие батчи не позволяют модели вычислять градиент на всем объёме данных.
С другой стороны, слишком большое значение может привести к нехватке памяти при обучении на больших объёмах данных и снижению эффективности обучения из-за того,
что для каждого шага обновления модели требуется больше времени на вычисление градиента.

Таким образом, одна эпоха может состоять из нескольких итераций обучения на разных батчах,
пока не будут обработаны все обучающие данные. Например, если обучающий набор содержит 50_000 записей,
и мы выбираем размер батча в 100, то в каждой эпохе будет 500 итераций обучения на разных батчах.
При этом в каждой итерации будут использоваться 100 записей.

In [63]:
table = [
    ['Количество входных нейронов:', n_inputs],
    ['Количество выходных нейронов:', n_outputs],
    ['Количество нейронов в скрытом слое:', n_hidden],
    ['Скорость обучения:', learning_rate],
    ['Количество эпох:', n_epochs],
    ['Количество образцов данных:', batch_size],
    ['X_train', X_train.shape],
    ['X_test', X_test.shape],
    ['y_train', y_train.shape],
    ['y_test', y_test.shape]
]

print(tabulate(table, headers=['Параметр', 'Значение'], tablefmt='pretty', colalign=('left', 'right')))

+-------------------------------------+-------------+
| Параметр                            |    Значение |
+-------------------------------------+-------------+
| Количество входных нейронов:        |         784 |
| Количество выходных нейронов:       |          10 |
| Количество нейронов в скрытом слое: |         522 |
| Скорость обучения:                  |         0.1 |
| Количество эпох:                    |         501 |
| Количество образцов данных:         |          32 |
| X_train                             | (6000, 784) |
| X_test                              | (2000, 784) |
| y_train                             |     (6000,) |
| y_test                              |     (2000,) |
+-------------------------------------+-------------+


# Алгоритм стохастического градиентного спуска (SGD)

Алгоритм стохастического градиентного спуска (SGD) - это оптимизационный алгоритм, который используется для обучения нейронных сетей.
Он основан на градиентном спуске, который является методом оптимизации для нахождения минимума функции.
Шаг оптимизации в алгоритме стохастического градиентного спуска вычисляется на основе градиента функции потерь на каждой итерации.
Однако, в отличие от обычного градиентного спуска, который вычисляет градиент по всем обучающим примерам одновременно,
SGD вычисляет градиенты на каждой итерации только для одного случайно выбранного обучающего примера.
Это позволяет обучению сети быть более быстрым и эффективным.

По сути, в алгоритме SGD мы делаем следующие шаги:

1) Выбираем случайный обучающий пример из обучающей выборки.
2) Прямое распространение (forward propagation): вычисляем выход модели на этом примере.
3) Обратное распространение (backpropagation): вычисляем градиенты функции потерь на выходном слое и скрытом слое.
4) Обновляем веса модели на основе вычисленных градиентов.
5) Повторяем шаги 1-4 для каждого обучающего примера в обучающей выборке.

In [ ]:
%%time
# Инициализация весов
weights_input_hidden = np.random.uniform(-0.5, 0.5, size=(n_inputs, n_hidden))  # инициализируем матрицу весов между входным и скрытым слоями случайными значениями из равномерного распределения в диапазоне [-0.5, 0.5].
bias_hidden = np.zeros(n_hidden) # инициализируем пороговые значения для скрытого слоя нулями.

weights_hidden_output = np.random.uniform(-0.5, 0.5, size=(n_hidden, n_outputs)) # инициализируем матрицу весов между скрытым и выходным слоями случайными значениями из равномерного распределения в диапазоне [-0.5, 0.5].
bias_output = np.zeros(n_outputs) # инициализируем пороговые значения для скрытого слоя нулями.

In [ ]:

# Обучение модели с помощью SGD, градиент вычисляется на небольших случайных подмножествах данных (батчах) в каждую эпоху.
for epoch in tqdm_notebook(range(n_epochs)):
    # Перемешиваем обучающие данные
    indices = np.random.permutation(len(X_train)) # Перемешиваем индексы обучающих данных в случайном порядке, чтобы модель обучалась на случайном наборе данных на каждой эпохе.
    X_train = X_train[indices]
    y_train = y_train[indices]

    for i in range(0, len(X_train), batch_size):
        # Получаем батч обучающих данных
        X_batch = X_train[i:i+batch_size] # Делим обучающие данные на батчи заданного размера.
        y_batch = y_train[i:i+batch_size] # Делим обучающие данные на батчи заданного размера.

        # Прямое распространение
        hidden_inputs = np.dot(X_batch, weights_input_hidden) + bias_hidden # получаем выходные значения скрытого слоя или сумму взвешенных входов для нейронов скрытого слоя.
        hidden_outputs = sigmoid(hidden_inputs) # получаем выходные значения скрытого  слоя

        output_inputs = np.dot(hidden_outputs, weights_hidden_output) + bias_output # вычисляем взвешенную сумму выходов скрытого слоя и получаем входные значения для выходного слоя.
        y_pred = softmax(output_inputs) # получаем предсказания вероятностей для каждого класса.

        # Обратное распространение ошибки
        error = y_pred - np.eye(n_outputs)[y_batch] # находим ошибку на выходном слое
        grad_output = error / len(X_batch) # вычисляем градиент функции потерь по выходному слою.
        grad_hidden = np.dot(grad_output, weights_hidden_output.T) * hidden_outputs * (1 - hidden_outputs) # вычисляем градиент функции потерь по скрытому слою.

        # Обновление весов и пороговых значений на скрытом и выходном слое
        weights_hidden_output -= learning_rate * np.dot(hidden_outputs.T, grad_output)
        bias_output -= learning_rate * np.sum(grad_output, axis=0)

        weights_input_hidden -= learning_rate * np.dot(X_batch.T, grad_hidden)
        bias_hidden -= learning_rate * np.sum(grad_hidden, axis=0)

    # Вычисление функции потерь на обучающих и тестовых данных
    hidden_train = np.dot(X_train, weights_input_hidden) + bias_hidden # Вычисляем значения скрытого слоя на обучающих данных
    hidden_train = sigmoid(hidden_train) # результат применения функции активации сигмоиды к выходному значению скрытого слоя.

    output_train = np.dot(hidden_train, weights_hidden_output) + bias_output # Вычисляем значения на выходном слое
    y_pred_train = softmax(output_train) # результат применения функции активации softmax к выходному значению выходного слоя, чтобы получить вероятности принадлежности к каждому классу.
    train_loss = np.mean(-np.log(y_pred_train[np.arange(len(X_train)), y_train])) # Рассчитываем значение функции потерь на обучающих данных с помощью кросс-энтропии

    hidden_test = np.dot(X_test, weights_input_hidden) + bias_hidden # Вычисляем значения скрытого слоя на тестовых данных
    hidden_test = sigmoid(hidden_test) # Применяем функцию активации sigmoid к скрытым значениям на тестовых данных

    output_test = np.dot(hidden_test, weights_hidden_output) + bias_output # Вычисляем значения на выходном слое для тестовых данных
    y_pred_test = softmax(output_test) # вычисляем предсказанные вероятности классов для тестовых данных
    test_loss = np.mean(-np.log(y_pred_test[np.arange(len(X_test)), y_test])) # вычисляем значение функции потерь на тестовой выборке с помощью кросс-энтропии

    # Выводим значение функции потерь на каждой эпохе
    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")

# [05:25<00:00, 12.261it/s] y_train = y_train[:6000] n_epochs = 500 Accuracy: 0.9155
# [28:05<00:00, 11.01it/s] y_train = y_train[:16000] n_epochs = 1000 Accuracy: 0.95225

'''
+-------------------------------------+--------------+
| Параметр                            |     Значение |
+-------------------------------------+--------------+
| Количество входных нейронов:        |          784 |
| Количество выходных нейронов:       |           10 |
| Количество нейронов в скрытом слое: |          522 |
| Скорость обучения:                  |          0.1 |
| Количество эпох:                    |         1501 |
| Количество образцов данных:         |           32 |
| X_train                             | (15000, 784) |
| X_test                              |  (5000, 784) |
| y_train                             |     (15000,) |
| y_test                              |      (5000,) |
+-------------------------------------+--------------+

Epoch 1450, Train Loss: 0.0002, Test Loss: 0.2152
Epoch 1500, Train Loss: 0.0002, Test Loss: 0.2157
Wall time: 49min 49s
Accuracy: 0.9524

+-------------------------------------+--------------+
| Параметр                            |     Значение |
+-------------------------------------+--------------+
| Количество входных нейронов:        |          784 |
| Количество выходных нейронов:       |           10 |
| Количество нейронов в скрытом слое: |          397 |
| Скорость обучения:                  |         0.05 |
| Количество эпох:                    |         1001 |
| Количество образцов данных:         |           32 |
| X_train                             | (60000, 784) |
| X_test                              | (10000, 784) |
| y_train                             |     (60000,) |
| y_test                              |     (10000,) |
+-------------------------------------+--------------+

Epoch 1000, Train Loss: 0.0004, Test Loss: 0.0820
Wall time: 1h 27min 1s
Accuracy: 0.9796



после Grid Search:
+-------------------------------------+--------------+
| Параметр                            |     Значение |
+-------------------------------------+--------------+
| Количество входных нейронов:        |          784 |
| Количество выходных нейронов:       |           10 |
| Количество нейронов в скрытом слое: |          522 |
| Скорость обучения:                  |          0.1 |
| Количество эпох:                    |         1501 |
| Количество образцов данных:         |           64 |
| X_train                             | (60000, 784) |
| X_test                              | (10000, 784) |
| y_train                             |     (60000,) |
| y_test                              |     (10000,) |
+-------------------------------------+--------------+

Epoch 1500, Train Loss: 0.0002, Test Loss: 0.0803
Wall time: 1h 46min 40s
Accuracy: 0.9809

'''

вывод значения функции потерь на обучающих и тестовых данных в конце каждой эпохи позволяет обнаружить переобучение модели,
когда функция потерь на обучающих данных продолжает уменьшаться, а функция потерь на тестовых данных начинает расти.

In [67]:
# Предсказание классов для новых данных
hidden_inputs = np.dot(X_test, weights_input_hidden) + bias_hidden
hidden_outputs = sigmoid(hidden_inputs)

output_inputs = np.dot(hidden_outputs, weights_hidden_output) + bias_output
y_pred = np.argmax(output_inputs, axis=1)

In [68]:
# Вычисление точности модели на тестовых данных
accuracy = np.mean(y_pred == y_test)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9155


## Grid Search для настройки параметров модели:

In [7]:
# Определение значений для параметров модели
n_inputs = X_train.shape[1]  # Количество входных нейронов
n_outputs = len(np.unique(y_train))  # Количество выходных нейронов

n_hidden_values = [int(X_train.shape[1]/2), int(X_train.shape[1]/3), int(X_train.shape[1]/1.5), X_train.shape[1]]  # Количество нейронов в скрытом слое [50, 100, 200]
learning_rate_values = [0.05, 0.1, 0.25]  # Скорость обучения
n_epochs_values = [500, 1000, 1500, 2000]  # Количество эпох обучения
batch_size_values = [16, 32, 64]  # Размер батча

In [8]:
# Инициализация переменных для хранения наилучших значений гиперпараметров и метрики качества
best_accuracy = 0
best_n_hidden = None
best_learning_rate = None
best_n_epochs = None
best_batch_size = None

In [9]:
%%time
# Перебор всех возможных комбинаций параметров
for n_hidden in n_hidden_values:
    for learning_rate in learning_rate_values:
        for n_epochs in n_epochs_values:
            for batch_size in batch_size_values:
                # Инициализация весов
                weights_input_hidden = np.random.uniform(-0.5, 0.5, size=(n_inputs, n_hidden))
                bias_hidden = np.zeros(n_hidden)

                weights_hidden_output = np.random.uniform(-0.5, 0.5, size=(n_hidden, n_outputs))
                bias_output = np.zeros(n_outputs)

                # Обучение модели с помощью SGD
                for epoch in range(n_epochs):
                    # Перемешиваем обучающие данные
                    indices = np.random.permutation(len(X_train))
                    X_train = X_train[indices]
                    y_train = y_train[indices]

                    for i in range(0, len(X_train), batch_size):
                        # Получаем батч обучающих данных
                        X_batch = X_train[i:i+batch_size]
                        y_batch = y_train[i:i+batch_size]

                        # Прямое распространение
                        hidden_inputs = np.dot(X_batch, weights_input_hidden) + bias_hidden
                        hidden_outputs = sigmoid(hidden_inputs)

                        output_inputs = np.dot(hidden_outputs, weights_hidden_output) + bias_output
                        y_pred = softmax(output_inputs)

                        # Обратное распространение ошибки
                        error = y_pred - np.eye(n_outputs)[y_batch]
                        grad_output = error / len(X_batch)
                        grad_hidden = np.dot(grad_output, weights_hidden_output.T) * hidden_outputs * (1 - hidden_outputs)
                        # Обновление весов и пороговых значений
                        weights_hidden_output -= learning_rate * np.dot(hidden_outputs.T, grad_output)
                        bias_output -= learning_rate * np.sum(grad_output, axis=0)

                        weights_input_hidden -= learning_rate * np.dot(X_batch.T, grad_hidden)
                        bias_hidden -= learning_rate * np.sum(grad_hidden, axis=0)
            # Вычисление точности модели на тестовых данных
            hidden_inputs = np.dot(X_test, weights_input_hidden) + bias_hidden
            hidden_outputs = sigmoid(hidden_inputs)

            output_inputs = np.dot(hidden_outputs, weights_hidden_output) + bias_output
            y_pred = np.argmax(output_inputs, axis=1)

            accuracy = np.mean(y_pred == y_test)

            # Обновление наилучших значений гиперпараметров и метрики качества
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_n_hidden = n_hidden
                best_learning_rate = learning_rate
                best_n_epochs = n_epochs
                best_batch_size = batch_size

            print(f"n_hidden={n_hidden}, learning_rate={learning_rate}, n_epochs={n_epochs}, batch_size={batch_size}, Accuracy: {accuracy:.4f}")
print(f"Best n_hidden: {best_n_hidden}")
print(f"Best learning_rate: {best_learning_rate}")
print(f"Best n_epochs: {best_n_epochs}")
print(f"Best batch_size: {best_batch_size}")
print(f"Best accuracy: {best_accuracy:.4f}")
'''
Best n_hidden: 522
Best learning_rate: 0.1
Best n_epochs: 1500
Best batch_size: 64
Best accuracy: 0.9550
Wall time: 5h 55min 41s
'''

n_hidden=392, learning_rate=0.05, n_epochs=500, batch_size=64, Accuracy: 0.9200
n_hidden=392, learning_rate=0.05, n_epochs=1000, batch_size=64, Accuracy: 0.9100
n_hidden=392, learning_rate=0.05, n_epochs=1500, batch_size=64, Accuracy: 0.9050
n_hidden=392, learning_rate=0.05, n_epochs=2000, batch_size=64, Accuracy: 0.9250
n_hidden=392, learning_rate=0.1, n_epochs=500, batch_size=64, Accuracy: 0.9200
n_hidden=392, learning_rate=0.1, n_epochs=1000, batch_size=64, Accuracy: 0.9300
n_hidden=392, learning_rate=0.1, n_epochs=1500, batch_size=64, Accuracy: 0.9300
n_hidden=392, learning_rate=0.1, n_epochs=2000, batch_size=64, Accuracy: 0.9300
n_hidden=392, learning_rate=0.25, n_epochs=500, batch_size=64, Accuracy: 0.9500
n_hidden=392, learning_rate=0.25, n_epochs=1000, batch_size=64, Accuracy: 0.9350
n_hidden=392, learning_rate=0.25, n_epochs=1500, batch_size=64, Accuracy: 0.9200
n_hidden=392, learning_rate=0.25, n_epochs=2000, batch_size=64, Accuracy: 0.9400
n_hidden=261, learning_rate=0.05, n

In [ ]:
# n_hidden_values = [50, 100, 200]  # Количество нейронов в скрытом слое
# learning_rate_values = [0.01, 0.1, 0.5]  # Скорость обучения
# n_epochs_values = [100, 500, 1000]  # Количество эпох обучения
# batch_size_values = [16, 32, 64]  # Размер батча
#
# n_hidden=50, learning_rate=0.01, n_epochs=100, batch_size=64, Accuracy: 0.6750
# n_hidden=50, learning_rate=0.01, n_epochs=500, batch_size=64, Accuracy: 0.8100
# n_hidden=50, learning_rate=0.01, n_epochs=1000, batch_size=64, Accuracy: 0.8900
# n_hidden=50, learning_rate=0.1, n_epochs=100, batch_size=64, Accuracy: 0.8650
# n_hidden=50, learning_rate=0.1, n_epochs=500, batch_size=64, Accuracy: 0.9350
# n_hidden=50, learning_rate=0.1, n_epochs=1000, batch_size=64, Accuracy: 0.9250
# n_hidden=50, learning_rate=0.5, n_epochs=100, batch_size=64, Accuracy: 0.9300
# n_hidden=50, learning_rate=0.5, n_epochs=500, batch_size=64, Accuracy: 0.9150
# n_hidden=50, learning_rate=0.5, n_epochs=1000, batch_size=64, Accuracy: 0.9350
# n_hidden=100, learning_rate=0.01, n_epochs=100, batch_size=64, Accuracy: 0.7600
# n_hidden=100, learning_rate=0.01, n_epochs=500, batch_size=64, Accuracy: 0.8900
# n_hidden=100, learning_rate=0.01, n_epochs=1000, batch_size=64, Accuracy: 0.8800
# n_hidden=100, learning_rate=0.1, n_epochs=100, batch_size=64, Accuracy: 0.8750
# n_hidden=100, learning_rate=0.1, n_epochs=500, batch_size=64, Accuracy: 0.9350
# n_hidden=100, learning_rate=0.1, n_epochs=1000, batch_size=64, Accuracy: 0.9150
# n_hidden=100, learning_rate=0.5, n_epochs=100, batch_size=64, Accuracy: 0.9200
# n_hidden=100, learning_rate=0.5, n_epochs=500, batch_size=64, Accuracy: 0.9000
# n_hidden=100, learning_rate=0.5, n_epochs=1000, batch_size=64, Accuracy: 0.9350
# n_hidden=200, learning_rate=0.01, n_epochs=100, batch_size=64, Accuracy: 0.7700
# n_hidden=200, learning_rate=0.01, n_epochs=500, batch_size=64, Accuracy: 0.8950
# n_hidden=200, learning_rate=0.01, n_epochs=1000, batch_size=64, Accuracy: 0.9200
# n_hidden=200, learning_rate=0.1, n_epochs=100, batch_size=64, Accuracy: 0.9050
# n_hidden=200, learning_rate=0.1, n_epochs=500, batch_size=64, Accuracy: 0.9100
# n_hidden=200, learning_rate=0.1, n_epochs=1000, batch_size=64, Accuracy: 0.9400
# n_hidden=200, learning_rate=0.5, n_epochs=100, batch_size=64, Accuracy: 0.9100
# n_hidden=200, learning_rate=0.5, n_epochs=500, batch_size=64, Accuracy: 0.9300
# n_hidden=200, learning_rate=0.5, n_epochs=1000, batch_size=64, Accuracy: 0.9250
# Best n_hidden: 200
# Best learning_rate: 0.1
# Best n_epochs: 1000
# Best batch_size: 64
# Best accuracy: 0.9400

In [ ]:
# n_hidden_values = [int(X_train.shape[1]/2), int(X_train.shape[1]/3), int(X_train.shape[1]/1.5), X_train.shape[1]]
# learning_rate_values = [0.05, 0.1, 0.25]  # Скорость обучения
# n_epochs_values = [500, 1000, 1500, 2000]  # Количество эпох обучения
# batch_size_values = [16, 32, 64]  # Размер батча

# n_hidden=392, learning_rate=0.05, n_epochs=500, batch_size=64, Accuracy: 0.9200
# n_hidden=392, learning_rate=0.05, n_epochs=1000, batch_size=64, Accuracy: 0.9100
# n_hidden=392, learning_rate=0.05, n_epochs=1500, batch_size=64, Accuracy: 0.9050
# n_hidden=392, learning_rate=0.05, n_epochs=2000, batch_size=64, Accuracy: 0.9250
# n_hidden=392, learning_rate=0.1, n_epochs=500, batch_size=64, Accuracy: 0.9200
# n_hidden=392, learning_rate=0.1, n_epochs=1000, batch_size=64, Accuracy: 0.9300
# n_hidden=392, learning_rate=0.1, n_epochs=1500, batch_size=64, Accuracy: 0.9300
# n_hidden=392, learning_rate=0.1, n_epochs=2000, batch_size=64, Accuracy: 0.9300
# n_hidden=392, learning_rate=0.25, n_epochs=500, batch_size=64, Accuracy: 0.9500
# n_hidden=392, learning_rate=0.25, n_epochs=1000, batch_size=64, Accuracy: 0.9350
# n_hidden=392, learning_rate=0.25, n_epochs=1500, batch_size=64, Accuracy: 0.9200
# n_hidden=392, learning_rate=0.25, n_epochs=2000, batch_size=64, Accuracy: 0.9400
# n_hidden=261, learning_rate=0.05, n_epochs=500, batch_size=64, Accuracy: 0.9100
# n_hidden=261, learning_rate=0.05, n_epochs=1000, batch_size=64, Accuracy: 0.9150
# n_hidden=261, learning_rate=0.05, n_epochs=1500, batch_size=64, Accuracy: 0.9300
# n_hidden=261, learning_rate=0.05, n_epochs=2000, batch_size=64, Accuracy: 0.9250
# n_hidden=261, learning_rate=0.1, n_epochs=500, batch_size=64, Accuracy: 0.9200
# n_hidden=261, learning_rate=0.1, n_epochs=1000, batch_size=64, Accuracy: 0.9350
# n_hidden=261, learning_rate=0.1, n_epochs=1500, batch_size=64, Accuracy: 0.9250
# n_hidden=261, learning_rate=0.1, n_epochs=2000, batch_size=64, Accuracy: 0.9050
# n_hidden=261, learning_rate=0.25, n_epochs=500, batch_size=64, Accuracy: 0.9250
# n_hidden=261, learning_rate=0.25, n_epochs=1000, batch_size=64, Accuracy: 0.9350
# n_hidden=261, learning_rate=0.25, n_epochs=1500, batch_size=64, Accuracy: 0.9200
# n_hidden=261, learning_rate=0.25, n_epochs=2000, batch_size=64, Accuracy: 0.9450
# n_hidden=522, learning_rate=0.05, n_epochs=500, batch_size=64, Accuracy: 0.9300
# n_hidden=522, learning_rate=0.05, n_epochs=1000, batch_size=64, Accuracy: 0.9200
# n_hidden=522, learning_rate=0.05, n_epochs=1500, batch_size=64, Accuracy: 0.9200
# n_hidden=522, learning_rate=0.05, n_epochs=2000, batch_size=64, Accuracy: 0.9250
# n_hidden=522, learning_rate=0.1, n_epochs=500, batch_size=64, Accuracy: 0.9250
# n_hidden=522, learning_rate=0.1, n_epochs=1000, batch_size=64, Accuracy: 0.9050
# n_hidden=522, learning_rate=0.1, n_epochs=1500, batch_size=64, Accuracy: 0.9550
# n_hidden=522, learning_rate=0.1, n_epochs=2000, batch_size=64, Accuracy: 0.9100
# n_hidden=522, learning_rate=0.25, n_epochs=500, batch_size=64, Accuracy: 0.9350
# n_hidden=522, learning_rate=0.25, n_epochs=1000, batch_size=64, Accuracy: 0.9400
# n_hidden=522, learning_rate=0.25, n_epochs=1500, batch_size=64, Accuracy: 0.9100
# n_hidden=522, learning_rate=0.25, n_epochs=2000, batch_size=64, Accuracy: 0.9250
# n_hidden=784, learning_rate=0.05, n_epochs=500, batch_size=64, Accuracy: 0.9200
# n_hidden=784, learning_rate=0.05, n_epochs=1000, batch_size=64, Accuracy: 0.9300
# n_hidden=784, learning_rate=0.05, n_epochs=1500, batch_size=64, Accuracy: 0.8950
# n_hidden=784, learning_rate=0.05, n_epochs=2000, batch_size=64, Accuracy: 0.9250
# n_hidden=784, learning_rate=0.1, n_epochs=500, batch_size=64, Accuracy: 0.9300
# n_hidden=784, learning_rate=0.1, n_epochs=1000, batch_size=64, Accuracy: 0.9250
# n_hidden=784, learning_rate=0.1, n_epochs=1500, batch_size=64, Accuracy: 0.9300
# n_hidden=784, learning_rate=0.1, n_epochs=2000, batch_size=64, Accuracy: 0.9100
# n_hidden=784, learning_rate=0.25, n_epochs=500, batch_size=64, Accuracy: 0.9050
# n_hidden=784, learning_rate=0.25, n_epochs=1000, batch_size=64, Accuracy: 0.9300
# n_hidden=784, learning_rate=0.25, n_epochs=1500, batch_size=64, Accuracy: 0.9150
# n_hidden=784, learning_rate=0.25, n_epochs=2000, batch_size=64, Accuracy: 0.9250
# Best n_hidden: 522
# Best learning_rate: 0.1
# Best n_epochs: 1500
# Best batch_size: 64
# Best accuracy: 0.9550
# Wall time: 5h 55min 41s